### Import

In [1]:
# 데이터 처리 
import pandas as pd
import numpy as np
# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import folium
# 파일 관리 
import os

## EDA

### Read Data

`-` 데이터 확인하기.

In [2]:
os.getcwd()

'/Users/oldcar/Documents/GitHub/2022-AnalysisCompetition-Subway'

In [3]:
for path, _, files in os.walk(os.getcwd() + '/Data'): # mac에선 /
    for filename in files:
        print(os.path.join(path, filename))

/Users/oldcar/Documents/GitHub/2022-AnalysisCompetition-Subway/Data/Found Data/공기질_승객수_통합.csv
/Users/oldcar/Documents/GitHub/2022-AnalysisCompetition-Subway/Data/Found Data/서울교통공사_월별승하차인원_20201231.csv
/Users/oldcar/Documents/GitHub/2022-AnalysisCompetition-Subway/Data/Found Data/좌표.csv
/Users/oldcar/Documents/GitHub/2022-AnalysisCompetition-Subway/Data/Found Data/서울교통공사_역사심도정보_20200929.csv
/Users/oldcar/Documents/GitHub/2022-AnalysisCompetition-Subway/Data/Found Data/서울교통공사_환기구정보_202006.csv
/Users/oldcar/Documents/GitHub/2022-AnalysisCompetition-Subway/Data/Found Data/서울교통공사_역사건축정보_20200331.csv
/Users/oldcar/Documents/GitHub/2022-AnalysisCompetition-Subway/Data/Given Data/passengers2020.csv
/Users/oldcar/Documents/GitHub/2022-AnalysisCompetition-Subway/Data/Given Data/data3.csv
/Users/oldcar/Documents/GitHub/2022-AnalysisCompetition-Subway/Data/Given Data/data2.csv
/Users/oldcar/Documents/GitHub/2022-AnalysisCompetition-Subway/Data/Given Data/data1.csv
/Users/oldcar/Documents/GitHub/20

`-` 데이터 가져오기. 

In [4]:
congestion=pd.read_csv('Data/Given Data/data1.csv') # 지하철 혼잡도 데이터
aircondition = pd.read_csv('Data/Given Data/data2.csv') # 지하철 공기질 데이터 
station = pd.read_csv('Data/Given Data/data3.csv') # 지하철 정보 데이터
passengers = pd.read_csv('Data/Given Data/passengers2020.csv') # 지하철 별 승객 수 데이터

### Preprocessing

#### congestion

In [5]:
congestion.head() #전처리 필요없음. 

,조사일자,호선,역번호,역명,구분,5:30,6:00,6:30,7:00,7:30,...,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,24:00:00,24:30:00
0,평일,1호선,150,서울역,상선,8.2,12.9,21.5,28.4,46.7,...,15.1,15.9,17.1,16.0,19.0,15.2,10.5,7.8,6.0,0.9
1,평일,1호선,150,서울역,하선,11.8,13.5,12.2,17.8,26.4,...,36.5,31.2,37.6,44.6,45.6,37.8,28.8,19.3,7.1,0.4
2,평일,1호선,151,시청,상선,7.7,11.2,19.6,24.4,36.2,...,18.7,18.3,20.7,22.0,24.0,18.7,13.7,10.0,6.9,1.0
3,평일,1호선,151,시청,하선,9.5,11.3,11.6,16.7,23.2,...,32.0,27.7,34.8,39.3,37.4,36.3,23.0,18.1,4.0,1.6
4,평일,1호선,152,종각,상선,6.3,8.1,15.5,16.6,22.7,...,25.8,24.2,30.0,31.9,34.0,26.8,18.8,12.5,9.1,1.2


In [6]:
print(congestion.isnull().sum(), congestion.shape) #널값이 있지만 해당 시각에 운행하지 않은 정보도 필요

조사일자           0
호선             0
역번호            0
역명             0
구분             0
5:30          32
6:00          10
6:30          10
7:00          10
7:30          10
8:00          10
8:30          10
9:00          10
9:30          10
10:00         10
10:30         10
11:00         10
11:30         10
12:00         10
12:30         10
13:00         10
13:30         10
14:00         10
14:30         10
15:00         10
15:30         10
16:00         10
16:30         10
17:00         10
17:30         10
18:00         10
18:30         10
19:00         10
19:30         10
20:00         10
20:30         10
21:00         10
21:30         10
22:00         10
22:30         10
23:00         10
23:30         22
24:00:00    1112
24:30:00    1112
dtype: int64 (1668, 44)


#### aircondition

In [7]:
aircondition.head()

,호선,역사명,미세먼지(PM10),이산화탄소(CO2),포름알데히드(HCHO),일산화탄소(CO)
0,1,서울역1,76.4,627,16.4,0.6
1,1,시청1,100.7,610,17.8,1.0
2,1,종각,99.8,687,20.5,0.9
3,1,종로3가1,105.3,646,20.6,1.3
4,1,종로5가,99.4,532,16.4,1.1


공기질 점수를 만들어야 하니 가중합을 사용해야한다. 

In [53]:
aircondition.drop_duplicates(['호선'])['호선'] #중복 제거 후 값 확인

0      1
10     2
47     3
79     4
100    5
151    6
189    7
237    8
Name: 호선, dtype: int64

In [54]:
line_num = [f'{i}' for i in range(1,9)]

# 역사명 뒤에 붙은 호선 제거

for i in range(len(aircondition)):
    if aircondition.loc[i]['역사명'][-1] in line_num:
        aircondition.loc[i,'역사명'] = aircondition.loc[i,'역사명'][:-1]
        
# 호선 값 뒤에 '호선' 붙이기

aircondition['호선'] = aircondition['호선'].apply(lambda x : str(x) + '호선')

# 열 이름 변경 

aircondition.columns = ['호선', '역명', '미세먼지', '이산화탄소', '포름알데히드', '일산화탄소']

# 공기질 점수 0~1 사이로 표준화 (minmax scaler)

score_df = aircondition.loc[:,'미세먼지':'일산화탄소']

aircondition.loc[:,'미세먼지':'일산화탄소'] = score_df.apply(lambda x : (x - x.min()) / (x.max() - x.min()), axis = 0)
aircondition['공기질점수'] = aircondition.loc[:,'미세먼지':'일산화탄소'].mean(axis = 1)
aircondition['공기질점수'] = (aircondition['공기질점수'] - aircondition['공기질점수'].min()) / (aircondition['공기질점수'].max() - aircondition['공기질점수'].min())

aircondition.head()


,호선,역명,미세먼지,이산화탄소,포름알데히드,일산화탄소,공기질점수
0,1호선,서울역,0.335559,0.623145,0.371166,0.166667,0.408431
1,1호선,시청,0.741235,0.572700,0.414110,0.500000,0.700221
2,1호선,종각,0.726210,0.801187,0.496933,0.416667,0.785164
3,1호선,종로3가,0.818030,0.679525,0.500000,0.750000,0.907447
4,1호선,종로5가,0.719533,0.341246,0.371166,0.583333,0.615350


#### Station

In [89]:
station = pd.read_csv('Data/Given Data/data3.csv') # 지하철 정보 데이터
station.head()

,연번,자치구,해당역(호선),역개수
0,1,강남구,삼성(2) 선릉(2) 역삼(2) 강남(2) 압구정(3) 신사(3) 매봉(3) 도곡(...,21
1,2,강동구,천호(5) 강동(5) 길동(5) 굽은다리(5) 명일(5) 고덕(5) 상일동(5) 둔...,14
2,3,강북구,수유(4) 미아(4) 미아사거리(4),3
3,4,강서구,방화(5) 개화산(5) 김포공항(5) 송정(5) 마곡(5) 발산(5) 우장산(5) ...,9
4,5,관악구,낙성대(2) 서울대입구(2) 봉천(2) 신림(2),4


In [90]:
# 연번 제거 

station.drop(['연번'], axis = 1, inplace = True) # inplace 원본변화

In [91]:
# 각 구에 속하는 역 쪼개기

station_ls = []
gu_ls= []

for i in range(len(station)):
    station_split = [x for x in station['해당역(호선)'].str.split()[i]]
    station_ls += station_split
    gu_ls += [f'{station["자치구"][i]}']* len(station_split)

line = [f'{i[-2]}호선' for i in station_ls]

In [96]:
# 데이터프레임으로 다시 만들기. 

station_ls = list(map(lambda x : x[:-3],station_ls))

station = pd.DataFrame({'호선' : line, '역명' : station_ls, '자치구' : gu_ls})

,호선,역명,자치구
0,2호선,삼성,강남구
1,2호선,선릉,강남구
2,2호선,역삼,강남구
3,2호선,강남,강남구
4,3호선,압구정,강남구


In [102]:
# 9호선 제거 (8호선까지 데이터가 있음)

line9_mask = station['호선'] != '9호선'
station = station[line9_mask]

station.head()

,호선,역명,자치구
0,2호선,삼성,강남구
1,2호선,선릉,강남구
2,2호선,역삼,강남구
3,2호선,강남,강남구
4,3호선,압구정,강남구


In [101]:
station[line9_mask] # 이렇게 하면 true 값 일치만 출력

,호선,역명,자치구
0,2호선,삼성,강남구
1,2호선,선릉,강남구
2,2호선,역삼,강남구
3,2호선,강남,강남구
4,3호선,압구정,강남구
...,...,...,...
291,7호선,장암,의정부시
292,5호선,미사,하남시
293,5호선,하남풍산,하남시
294,5호선,하남시청,하남시
